In [0]:
#!kill -9 -1

In [95]:
!pip install keras --upgrade
!pip install keras_applications --upgrade
#!pip install Keras-Applications --upgrade
#!pip install tensorflow --upgrade

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.2.0)
Requirement not upgraded as not directly required: keras-preprocessing==1.0.1 in /usr/local/lib/python3.6/dist-packages (from keras) (1.0.1)
Requirement not upgraded as not directly required: scipy>=0.14 in /usr/local/lib/python3.6/dist-packages (from keras) (0.19.1)
Requirement not upgraded as not directly required: six>=1.9.0 in /usr/local/lib/python3.6/dist-packages (from keras) (1.11.0)
  Using cached https://files.pythonhosted.org/packages/e2/60/c557075e586e968d7a9c314aa38c236b37cb3ee6b37e8d57152b1a5e0b47/Keras_Applications-1.0.2-py2.py3-none-any.whl
Requirement not upgraded as not directly required: numpy>=1.9.1 in /usr/local/lib/python3.6/dist-packages (from keras) (1.14.5)
Requirement not upgraded as not directly required: h5py in /usr/local/lib/python3.6/dist-packages (from keras) (2.8.0)
Requirement not upgraded as not directly required: pyyaml in /usr/local/lib/python3.6/dist-packages (fr

## Загрузка данных


In [21]:
!pip install kaggle

    100% |████████████████████████████████| 51kB 20.3MB/s 
  Running setup.py bdist_wheel for kaggle ... - done
  Stored in directory: /content/.cache/pip/wheels/61/d6/e6/38fc0e2316f49b91aafdaf9a8d7756a9906b2ad7e501c1e165
Successfully built kaggle


In [0]:
!mkdir ~/.kaggle
!echo '{"username":"1111111111","key":"1111111111"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


In [32]:
!kaggle config set -n path -v /content
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

path is now set to: /content
test.zip: Skipping, found more recently modified local copy (use --force to force download)
train.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [45]:
!apt install tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 7 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu artful/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (630 kB/s)

78Selecting previously unselected package tree.
(Reading database ... 18396 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 16%] [#########.................................................] 8Unpacking tree (1.7.0-5) ...
7Progress: [ 33%] [###################.......................................] 87Progress: [ 50%] [#############################.............................] 8Setting up tree (1.7.0-5) ...
7P

In [0]:
#!mkdir ./data
!unzip -q ./competitions/dogs-vs-cats-redux-kernels-edition/train.zip -d ./data/
!unzip -q ./competitions/dogs-vs-cats-redux-kernels-edition/test.zip -d ./data/test

In [105]:
!mkdir ./data/train/dogs
!mkdir ./data/train/cats
!mv ./data/train/dog*.* ./data/train/dogs
!mv ./data/train/cat*.* ./data/train/cats

mkdir: cannot create directory ‘./data/train/dogs’: File exists
mkdir: cannot create directory ‘./data/train/cats’: File exists


In [106]:
!tree ./data -d

./data
├── test
│   └── test
└── train
    ├── cats
    └── dogs

5 directories


In [0]:
import numpy as np
import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Flatten
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array


In [0]:
IMG_SIZE = (224, 224)
batch_size = 32
epochs = 30

Аугментацию данных будем проводить с помощью ImageDataGenerator. Размер картинок приводим к (224, 224)

In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
#        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

In [130]:
train_generator = train_datagen.flow_from_directory(
        './data/train', 
        target_size=(IMG_SIZE[0], IMG_SIZE[1]), 
        batch_size=batch_size,
        class_mode='binary',
        subset = 'training')

validation_generator = train_datagen.flow_from_directory(
        './data/train', 
        target_size=(IMG_SIZE[0], IMG_SIZE[1]), 
        batch_size=batch_size,
        class_mode='binary',
        subset = 'validation')

test_generator = test_datagen.flow_from_directory(
        './data/test',
        target_size=(IMG_SIZE[0], IMG_SIZE[1]),
        batch_size=1,
        class_mode=None,
        classes=None,
        shuffle=False)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


## Обучение MobileNet

В качестве базовой модели берем MobileNet.
Размораживаем последний сверточный слой.
И добавляем два полносвязных

In [0]:
base_model = MobileNet(include_top = False,
                   weights = 'imagenet',
                   input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3),
                   pooling = 'avg')

In [0]:
for layer in base_model.layers[:93]:
    layer.trainable = False

In [137]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 114, 114, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
__________

In [120]:
base_model.output_shape[1:]

(1024,)

In [0]:
top_model = Sequential()
    
top_model.add(Dense(256, activation='relu', input_shape=base_model.output_shape[1:]))
top_model.add(Dropout(0.5))

top_model.add(Dense(128, activation='relu'))
top_model.add(Dropout(0.5))    

top_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [139]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 114, 114, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
__________

In [140]:
lr_reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                              factor=0.1,
                                              epsilon=1e-5, 
                                              patience=5, 
                                              verbose=1)

checkpoint = keras.callbacks.ModelCheckpoint('top_weights2.h5',
                                             save_best_only=True,
                                             verbose=1)



/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:928: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` insted.
  feed_dict[K.learning_phase()] = False


In [143]:
model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs=epochs,
        validation_data=validation_generator,
        callbacks=[lr_reduce, checkpoint])

Epoch 1/30
10/10 [==============================] - 81s 8s/step - loss: 0.5634 - acc: 0.7375 - val_loss: 0.4256 - val_acc: 0.7862

Epoch 00001: val_loss improved from inf to 0.42559, saving model to top_weights2.h5
Epoch 2/30
10/10 [==============================] - 75s 7s/step - loss: 0.3418 - acc: 0.8562 - val_loss: 0.5011 - val_acc: 0.7908

Epoch 00002: val_loss did not improve from 0.42559
Epoch 3/30
10/10 [==============================] - 77s 8s/step - loss: 0.2119 - acc: 0.9219 - val_loss: 0.1542 - val_acc: 0.9352

Epoch 00003: val_loss improved from 0.42559 to 0.15417, saving model to top_weights2.h5
Epoch 4/30
10/10 [==============================] - 77s 8s/step - loss: 0.2769 - acc: 0.8906 - val_loss: 0.6001 - val_acc: 0.8054

Epoch 00004: val_loss did not improve from 0.15417
Epoch 5/30
10/10 [==============================] - 76s 8s/step - loss: 0.2642 - acc: 0.8969 - val_loss: 0.3280 - val_acc: 0.8684

Epoch 00005: val_loss did not improve from 0.15417
Epoch 6/30
10/10 [==

10/10 [==============================] - 75s 7s/step - loss: 0.2154 - acc: 0.9375 - val_loss: 0.2271 - val_acc: 0.9032

Epoch 00010: val_loss did not improve from 0.12689
Epoch 11/30
10/10 [==============================] - 75s 7s/step - loss: 0.1790 - acc: 0.9250 - val_loss: 0.3182 - val_acc: 0.8560

Epoch 00011: val_loss did not improve from 0.12689
Epoch 12/30
10/10 [==============================] - 74s 7s/step - loss: 0.1171 - acc: 0.9531 - val_loss: 0.2669 - val_acc: 0.8758

Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00012: val_loss did not improve from 0.12689
Epoch 13/30
10/10 [==============================] - 74s 7s/step - loss: 0.1290 - acc: 0.9625 - val_loss: 0.2547 - val_acc: 0.8804

Epoch 00013: val_loss did not improve from 0.12689
Epoch 14/30
10/10 [==============================] - 76s 8s/step - loss: 0.1368 - acc: 0.9375 - val_loss: 0.2518 - val_acc: 0.8836

Epoch 00014: val_loss did not improve from 0.12689
Epoch 15/30
10/

10/10 [==============================] - 75s 7s/step - loss: 0.1545 - acc: 0.9250 - val_loss: 0.2572 - val_acc: 0.8836

Epoch 00020: val_loss did not improve from 0.12689
Epoch 21/30
10/10 [==============================] - 76s 8s/step - loss: 0.1247 - acc: 0.9656 - val_loss: 0.2551 - val_acc: 0.8844

Epoch 00021: val_loss did not improve from 0.12689
Epoch 22/30
10/10 [==============================] - 76s 8s/step - loss: 0.1229 - acc: 0.9437 - val_loss: 0.2575 - val_acc: 0.8824

Epoch 00022: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.

Epoch 00022: val_loss did not improve from 0.12689
Epoch 23/30
10/10 [==============================] - 74s 7s/step - loss: 0.1084 - acc: 0.9531 - val_loss: 0.2477 - val_acc: 0.8852

Epoch 00023: val_loss did not improve from 0.12689
Epoch 24/30
10/10 [==============================] - 78s 8s/step - loss: 0.1602 - acc: 0.9313 - val_loss: 0.2485 - val_acc: 0.8878

Epoch 00024: val_loss did not improve from 0.12689
Epoch 25/30
10/

10/10 [==============================] - 83s 8s/step - loss: 0.0907 - acc: 0.9656 - val_loss: 0.2374 - val_acc: 0.8918

Epoch 00030: val_loss did not improve from 0.12689


Сохраняем модель и веса

In [0]:
model.save("model.h5")

In [0]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [146]:
file_name = 'top_weights2.h5'

from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': file_name,
#  'mimeType': 'text/plain'
}
media = MediaFileUpload(file_name, 
#                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1pubf_uQO3sKeLyGwNkOtc7axJTsk0Y11


## Предсказание и сабмит

In [0]:
model.load_weights(filepath='top_weights2.h5')

In [0]:
# загружаем входное изображение и предобрабатываем
def load_image(path, target_size=IMG_SIZE):
    img = load_img(path, target_size=target_size)  # загрузка и масштабирование изображения
    array = img_to_array(img)
    return preprocess_input(array)  # предобработка для MobileNet
def predict_generator(files):
    while True:
        for path in files:
            yield np.array([load_image(path)])

In [0]:
import re
from glob import glob

test_files = glob('./data/test/test/*.jpg')

In [162]:
pred = model.predict_generator(predict_generator(test_files), len(test_files), max_queue_size=500, verbose=1)

12500/12500 [==============================] - 167s 13ms/step


In [163]:
train_generator.class_indices

{'cats': 0, 'dogs': 1}

In [164]:
pred.shape

(12500, 1)

In [0]:


with open('submit.txt', 'w') as dst:
    dst.write('id,label\n')
    for path, score in zip(test_files, pred):
        dst.write('%s,%f\n' % (re.search('(\d+)', path).group(0), score))

In [0]:
test_files

In [167]:
!head submit.txt

id,label
9506,0.000011
7955,0.000056
10702,0.999776
4862,0.802012
8436,0.001531
8921,0.000236
802,0.002098
10462,0.998887
5730,0.968289


In [168]:
file_name = 'submit.txt'

from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': file_name,
#  'mimeType': 'text/plain'
}
media = MediaFileUpload(file_name, 
#                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1dXJmfqIVQ7xCQVW0gFadN0WbESd2HO4V


public score: 0.10257